In [1]:
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import os
import logging

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
path_data_folder = Path.cwd().parent.parent / 'data'
path_data_folder

PosixPath('/home/tim/Documents/feat-store/data')

In [3]:
folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

In [4]:
df = pd.read_csv(folder_interim_data_milling / "milling_0.csv")
print(df.shape)
df_orig = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip')

(1157, 4507)


In [5]:
df_orig.head()

,cut_id,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [6]:
df.head()

,cut_id,vib_table__time_reversal_asymmetry_statistic__lag_1,vib_table__time_reversal_asymmetry_statistic__lag_2,vib_table__time_reversal_asymmetry_statistic__lag_3,vib_table__c3__lag_1,vib_table__c3__lag_2,vib_table__c3__lag_3,vib_table__cid_ce__normalize_True,vib_table__cid_ce__normalize_False,vib_table__symmetry_looking__r_0.0,...,vib_spindle__permutation_entropy__dimension_5__tau_1,vib_spindle__permutation_entropy__dimension_6__tau_1,vib_spindle__permutation_entropy__dimension_7__tau_1,vib_spindle__query_similarity_count__query_None__threshold_0.0,"vib_spindle__matrix_profile__feature_""min""__threshold_0.98","vib_spindle__matrix_profile__feature_""max""__threshold_0.98","vib_spindle__matrix_profile__feature_""mean""__threshold_0.98","vib_spindle__matrix_profile__feature_""median""__threshold_0.98","vib_spindle__matrix_profile__feature_""25""__threshold_0.98","vib_spindle__matrix_profile__feature_""75""__threshold_0.98"
0,0_0,0.059820,0.019726,0.090884,10.242404,9.998320,9.917406,8.316355,2.330515,0.0,...,3.528049,3.889565,3.964837,NaN,1.367334,2.665270,1.862989,1.863036,1.561820,2.097634
1,0_1,-0.032651,0.114725,-0.076326,10.013332,9.918871,10.131679,8.968916,2.595267,0.0,...,3.684120,3.951186,4.012640,NaN,0.909485,2.750737,1.812363,1.724483,1.530175,2.259445
2,0_10,0.062789,0.124616,0.091206,9.749092,9.611839,9.683704,8.938175,2.411882,0.0,...,3.582979,3.857200,3.955815,NaN,0.955919,2.509319,1.498419,1.466404,1.262464,1.646470
3,0_11,-0.041695,-0.085356,-0.074667,9.335468,9.284036,9.482934,8.809535,2.613744,0.0,...,3.661015,3.927690,4.036541,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0_12,0.076698,0.140736,0.045914,9.450062,9.291278,9.362848,8.643183,2.258746,0.0,...,3.666678,3.904193,4.036541,NaN,0.810572,1.879254,1.327153,1.330487,1.216766,1.457333


In [7]:
df_orig[['cut_id', 'tool_class']].drop_duplicates().head()

,cut_id,tool_class
0,0_0,0
64,0_1,0
128,0_2,0
192,0_3,0
256,0_4,0


In [8]:
# merge the two dataframes on the unique cut_id to get the tool_class in the df dataframe
df = df.merge(df_orig[['cut_id', 'case', 'tool_class']].drop_duplicates(), on='cut_id', how='left')
df.shape

(1157, 4509)

In [10]:
df

,cut_id,vib_table__time_reversal_asymmetry_statistic__lag_1,vib_table__time_reversal_asymmetry_statistic__lag_2,vib_table__time_reversal_asymmetry_statistic__lag_3,vib_table__c3__lag_1,vib_table__c3__lag_2,vib_table__c3__lag_3,vib_table__cid_ce__normalize_True,vib_table__cid_ce__normalize_False,vib_table__symmetry_looking__r_0.0,...,vib_spindle__permutation_entropy__dimension_7__tau_1,vib_spindle__query_similarity_count__query_None__threshold_0.0,"vib_spindle__matrix_profile__feature_""min""__threshold_0.98","vib_spindle__matrix_profile__feature_""max""__threshold_0.98","vib_spindle__matrix_profile__feature_""mean""__threshold_0.98","vib_spindle__matrix_profile__feature_""median""__threshold_0.98","vib_spindle__matrix_profile__feature_""25""__threshold_0.98","vib_spindle__matrix_profile__feature_""75""__threshold_0.98",case,tool_class
0,0_0,0.059820,0.019726,0.090884,10.242404,9.998320,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.665270,1.862989,1.863036,1.561820,2.097634,0,0
1,0_1,-0.032651,0.114725,-0.076326,10.013332,9.918871,10.131679,8.968916,2.595267,0.0,...,4.012640,NaN,0.909485,2.750737,1.812363,1.724483,1.530175,2.259445,0,0
2,0_10,0.062789,0.124616,0.091206,9.749092,9.611839,9.683704,8.938175,2.411882,0.0,...,3.955815,NaN,0.955919,2.509319,1.498419,1.466404,1.262464,1.646470,0,0
3,0_11,-0.041695,-0.085356,-0.074667,9.335468,9.284036,9.482934,8.809535,2.613744,0.0,...,4.036541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,0_12,0.076698,0.140736,0.045914,9.450062,9.291278,9.362848,8.643183,2.258746,0.0,...,4.036541,NaN,0.810572,1.879254,1.327153,1.330487,1.216766,1.457333,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,9_68,-0.001917,-0.003655,0.006743,0.932486,0.925402,0.941587,10.809945,0.986769,0.0,...,4.036541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1
1153,9_69,0.007929,0.017782,0.013091,1.006610,1.002354,1.020924,10.021115,1.075250,0.0,...,4.060443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1
1154,9_7,0.005498,0.007603,0.012817,0.754884,0.747748,0.759572,9.449920,0.990703,0.0,...,4.036541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1
1155,9_8,-0.011625,-0.013802,-0.010485,0.733736,0.727515,0.741080,9.285734,0.989809,0.0,...,3.988738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,1


In [32]:
df_orig[['cut_id', 'time', 'tool_class']]

,cut_id,time,tool_class
0,0_0,0.000,0
1,0_0,0.004,0
2,0_0,0.008,0
3,0_0,0.012,0
4,0_0,0.016,0
...,...,...,...
740475,166_65,0.236,1
740476,166_65,0.240,1
740477,166_65,0.244,1
740478,166_65,0.248,1


In [33]:
df.head()

,cut_id,vib_table__time_reversal_asymmetry_statistic__lag_1,vib_table__time_reversal_asymmetry_statistic__lag_2,vib_table__time_reversal_asymmetry_statistic__lag_3,vib_table__c3__lag_1,vib_table__c3__lag_2,vib_table__c3__lag_3,vib_table__cid_ce__normalize_True,vib_table__cid_ce__normalize_False,vib_table__symmetry_looking__r_0.0,...,vib_spindle__permutation_entropy__dimension_7__tau_1,vib_spindle__query_similarity_count__query_None__threshold_0.0,"vib_spindle__matrix_profile__feature_""min""__threshold_0.98","vib_spindle__matrix_profile__feature_""max""__threshold_0.98","vib_spindle__matrix_profile__feature_""mean""__threshold_0.98","vib_spindle__matrix_profile__feature_""median""__threshold_0.98","vib_spindle__matrix_profile__feature_""25""__threshold_0.98","vib_spindle__matrix_profile__feature_""75""__threshold_0.98",time,tool_class
0,0_0,0.05982,0.019726,0.090884,10.242404,9.99832,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.66527,1.862989,1.863036,1.56182,2.097634,0.000,0
1,0_0,0.05982,0.019726,0.090884,10.242404,9.99832,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.66527,1.862989,1.863036,1.56182,2.097634,0.004,0
2,0_0,0.05982,0.019726,0.090884,10.242404,9.99832,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.66527,1.862989,1.863036,1.56182,2.097634,0.008,0
3,0_0,0.05982,0.019726,0.090884,10.242404,9.99832,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.66527,1.862989,1.863036,1.56182,2.097634,0.012,0
4,0_0,0.05982,0.019726,0.090884,10.242404,9.99832,9.917406,8.316355,2.330515,0.0,...,3.964837,NaN,1.367334,2.66527,1.862989,1.863036,1.56182,2.097634,0.016,0


In [9]:
df.shape

(74048, 4509)

In [4]:
def read_csv(filename):
    'converts a filename to a pandas dataframe'
    return pd.read_csv(filename)


def main(folder_interim_data):
    """Runs data processing scripts to turn raw data from (../raw) into
    cleaned data ready to be analyzed (saved in ../processed).
    """
    logger = logging.getLogger(__name__)
    logger.info("making final data set from raw data")

    # get a list of file names
    files = os.listdir(folder_interim_data)
    file_list = [
        Path(folder_interim_data) / filename
        for filename in files
        if filename.endswith(".csv")
    ]

    # set up your pool
    with Pool(processes=2) as pool:  # or whatever your hardware can support

        # have your pool map the file names to dataframes
        df_list = pool.map(read_csv, file_list)

        # reduce the list of dataframes to a single dataframe
        combined_df = pd.concat(df_list, ignore_index=True)

        return combined_df

In [5]:
if __name__ == "__main__":
    log_fmt = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    logging.basicConfig(level=logging.INFO, format=log_fmt)

    df = main(folder_interim_data_milling)
    print("Final df shape:", df.shape)


#     df["births"] = np.ones(df.shape[0])
#     df.to_csv(project_dir / "data/processed" / "birth_geo_ind_apgar.csv.gz", compression="gzip", index=False)

#     # create a birth count for each unique geo and date
#     # this should reduce the size of the df significantly
#     df = df.drop(columns=["apgar5"]) # not needed in consolidated table
#     df = (
#         df.groupby(list(df.columns)[:-1], as_index=False)
#         .count()
#         .sort_values(by=["dob_yy", "dob_mm"])
#     )
#     print("Shape after consolidated birth count:", df.shape)

#     df.to_csv(project_dir / "data/processed" / "birth_geo_consolidated.csv.gz", compression="gzip", index=False)

2021-12-03 13:54:25,568 - __main__ - INFO - making final data set from raw data


Final df shape: (4628, 4506)


In [18]:
df.shape

(4628, 4506)

In [19]:
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,smcac__permutation_entropy__dimension_5__tau_1,smcac__permutation_entropy__dimension_6__tau_1,smcac__permutation_entropy__dimension_7__tau_1,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98"
0,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,1.0,...,2.383786,2.775729,3.094093,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248
1,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,1.0,...,2.149120,2.524759,2.883246,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660
2,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,1.0,...,2.358684,2.772571,3.142517,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001
3,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,1.0,...,2.232165,2.629432,2.953951,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085
4,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,1.0,...,2.220301,2.641679,3.023318,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327


In [9]:
df['time']

KeyError: 'time'

In [7]:
dfr = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
dfr.head()

,cut_id,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0
